# スコアシートの総当たり閾値検索に加えて，相関係数も算出する
## なぜ？
## ＝閾値をクリアした同一comのモデルの中で，最終的に使うモデルを一つに絞るため

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt

import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
#自作のモジュールのインポート
sys.path.append("..")
import module.master as master
import module.modeling_scores as modeling


### 方針
### ＝４つの区間をすべて使う，閾値の検索は無しとして決めのパラメータだけで絞り，その後各変数とfinal_gainとの相関係数を算出して，最終的に標準化した偏差値と組み合わせる

In [13]:
# 芦屋だけでひとまず機能を完成させる
#きめのパラメータ
place_name='asiya'
plus_m_th=3
num_hit_th=10
versions=['V4_1','V4_2','V4_3']
modes=['asiya_model_score_','asiya_model_score_period_']#モードは直近三カ月と，昨年同月の二種類
mode_types=['recent','period']#モードは直近三カ月と，昨年同月の二種類,modesと一緒にzipで回す

#for version in versions:
version='V4_2'
version_df=pd.DataFrame()#区間ごとのスコアシートの結合先（バージョンごとのまとめ）
for mode,mode_type in zip(modes,mode_types):
    for i in range(4):
        now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
        dir_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
        model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
        model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
         #決めのパラメータでいったん切り落とし
        model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
        model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる，大体一カ月に一件くらいをイメージ
        version_df= pd.concat([version_df, model_score_df], axis=0)#きめのパラメータで切り落としたので結合
#相関係数の算出（範囲は-1～1）
version_corr_df=version_df.corr()

In [14]:
version_corr_df['gain_final']

target_com               0.237996
depth                   -0.001983
target_per              -0.016965
threshold               -0.035868
total_get_test          -0.050708
total_use_test           0.025726
num_com_test            -0.053928
num_pred_test            0.025726
gain_test               -0.090430
gain_std_test            0.188157
num_hit_test            -0.168868
buy_hit_per_test        -0.231643
buy_hit_per_std_test    -0.168131
plus_month_num_test      0.004838
diff_mea_med_test       -0.128476
total_get_final          0.728426
total_use_final         -0.047977
num_com_final            0.044912
num_pred_final          -0.047977
gain_final               1.000000
gain_std_final           0.714854
num_hit_final            0.161411
buy_hit_per_final        0.263522
buy_hit_per_std_final    0.071783
plus_month_num_final     0.745362
diff_mea_med_final       0.230377
Name: gain_final, dtype: float64

In [4]:
model_score_df.to_csv("ex.csv")